In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
# ----------------------------
# CELL 0 — Title & Basic Info
# ----------------------------
# MealCheckAI - Kaggle Notebook (MAX-PRO Edition)
# Author: (Your Name)
# Purpose: Full Meal planning agent for clinical recovery scenarios (e.g., hepatitis A)
# Note: Optional Gemini integration (via GOOGLE_API_KEY secret). If not present,
#       the notebook uses deterministic local reasoning.
print("MealCheckAI Kaggle Notebook (MAX-PRO Edition)")
print("Run cells sequentially. Optional: add GOOGLE_API_KEY via Kaggle Add-ons -> Secrets")


MealCheckAI Kaggle Notebook (MAX-PRO Edition)
Run cells sequentially. Optional: add GOOGLE_API_KEY via Kaggle Add-ons -> Secrets


In [42]:
# ----------------------------
# CELL 1 — Install lightweight packages (if missing)
# ----------------------------
# We use only small libraries: pydantic (for models), tinydb (for embedded JSON DB), and rich for nicer prints.
# These are optional but recommended. Kaggle usually has pydantic installed.
import sys
import subprocess

def pip_install(pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs], stdout=subprocess.DEVNULL)
    except Exception as e:
        print("Package install error (non-fatal):", e)

# Install tinydb and rich if not present
pip_install(["tinydb", "rich"])

print("Packages installation step complete (if needed).")


Packages installation step complete (if needed).


In [43]:
# ----------------------------
# CELL 2 — Imports & Secrets
# ----------------------------
import os, json, sqlite3, datetime, uuid, math, textwrap
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field, asdict
from rich import print as rprint
from rich.table import Table
from rich.console import Console
console = Console()

# Try to read GOOGLE_API_KEY from Kaggle secrets (optional)
GOOGLE_API_KEY = None
try:
    # Kaggle secrets are available via environment variable if you added them
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") or os.environ.get("GEMINI_API_KEY") or os.environ.get("GOOGLE_API_KEY_SECRET")
except Exception:
    GOOGLE_API_KEY = None

if not GOOGLE_API_KEY:
    console.print("[yellow]No GOOGLE_API_KEY found in environment. Gemini integration disabled by default.[/yellow]")
else:
    console.print("[green]GOOGLE_API_KEY found — Gemini integration enabled (optional).[/green]")

# Local DB file for sessions/memory
DB_FILE = "mealcheckai.db"


GOOGLE_API_KEY found — Gemini integration enabled (optional).

In [44]:
# ----------------------------
# CELL 3 — Persistent Storage (SQLite) Setup
# ----------------------------
def init_db(db_file=DB_FILE):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    # Patients table
    c.execute("""
        CREATE TABLE IF NOT EXISTS patients (
            id TEXT PRIMARY KEY,
            name TEXT,
            age INTEGER,
            gender TEXT,
            conditions TEXT, -- JSON list
            allergies TEXT,   -- JSON list
            calorie_target INTEGER,
            max_daily_prep_minutes INTEGER,
            created_at TEXT
        )
    """)
    # Pantry table
    c.execute("""
        CREATE TABLE IF NOT EXISTS pantry (
            id TEXT PRIMARY KEY,
            patient_id TEXT,
            item TEXT,
            quantity TEXT,
            notes TEXT,
            added_at TEXT
        )
    """)
    # Recipes table (prepopulated small KB)
    c.execute("""
        CREATE TABLE IF NOT EXISTS recipes (
            id TEXT PRIMARY KEY,
            title TEXT,
            ingredients TEXT, -- JSON list of strings
            instructions TEXT,
            calories INTEGER,
            prep_minutes INTEGER,
            difficulty TEXT,
            condition_flags TEXT -- JSON list like ["hepatitis_a_avoid", ...]
        )
    """)
    # Plans table
    c.execute("""
        CREATE TABLE IF NOT EXISTS plans (
            id TEXT PRIMARY KEY,
            patient_id TEXT,
            plan_json TEXT,
            doctor_approved INTEGER DEFAULT 0,
            created_at TEXT
        )
    """)
    # Memory table
    c.execute("""
        CREATE TABLE IF NOT EXISTS memory (
            id TEXT PRIMARY KEY,
            patient_id TEXT,
            key TEXT,
            value TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    return conn

conn = init_db()
console.print("[green]Initialized SQLite DB and tables.[/green]")


Initialized SQLite DB and tables.

In [45]:
# ----------------------------
# CELL 4 — Small Recipe Knowledge Base (Real, curated)
# ----------------------------
# We'll insert a curated small recipe database suitable for recovery diets (hepatitis A friendly).
# These are "real" recipes written out (not mocked outputs) but simplified for notebook usage.

import uuid, json
def insert_recipe(conn, title, ingredients, instructions, calories, prep_minutes, difficulty, condition_flags):
    c = conn.cursor()
    rid = str(uuid.uuid4())
    c.execute("INSERT OR REPLACE INTO recipes (id, title, ingredients, instructions, calories, prep_minutes, difficulty, condition_flags) VALUES (?,?,?,?,?,?,?,?)",
              (rid, title, json.dumps(ingredients), instructions, calories, prep_minutes, difficulty, json.dumps(condition_flags)))
    conn.commit()
    return rid

# Example recipes (suitable for sensitive digestion & hepatitis A friendly)
recipes_to_add = [
    {
        "title": "Plain Boiled Rice with Steamed Carrots and Poached Chicken",
        "ingredients": ["rice (1 cup)", "chicken breast (120g)", "carrots (1 cup sliced)", "salt (small pinch)", "water"],
        "instructions": "Boil rice with 2 cups water until soft. Steam sliced carrots until tender. Poach chicken breast in simmering water until cooked through; shred. Serve mildly salted.",
        "calories": 450,
        "prep_minutes": 30,
        "difficulty": "easy",
        "condition_flags": ["low_fat", "soft_food", "no_raw", "mild_seasoning"]
    },
    {
        "title": "Oatmeal with Banana and Honey (soft)",
        "ingredients": ["rolled oats (1/2 cup)", "water or milk (1 cup)", "banana (1 small mashed)", "honey (1 tsp)"],
        "instructions": "Cook oats with water until very soft. Stir mashed banana and a little honey. Serve warm.",
        "calories": 320,
        "prep_minutes": 10,
        "difficulty": "easy",
        "condition_flags": ["soft_food", "easy_to_digest"]
    },
    {
        "title": "Vegetable Clear Soup (Carrot, Potato, Zucchini)",
        "ingredients": ["carrot (1)", "potato (1 small)", "zucchini (1/2)", "water (4 cups)", "salt (small pinch)"],
        "instructions": "Simmer chopped vegetables in water until soft. Blend very lightly or strain to keep clear. Season very mildly.",
        "calories": 150,
        "prep_minutes": 25,
        "difficulty": "easy",
        "condition_flags": ["liquid", "hydration", "low_fat", "soft_food"]
    },
    {
        "title": "Steamed Fish (mild) with Mashed Potato",
        "ingredients": ["white fish fillet (100g)", "potato (1)", "butter (small 1 tsp)", "salt (tiny)"],
        "instructions": "Steam fish until flaky. Boil potato and mash with tiny butter. Serve fish flaked over mashed potato.",
        "calories": 360,
        "prep_minutes": 25,
        "difficulty": "medium",
        "condition_flags": ["low_fat", "no_raw"]
    },
]

for r in recipes_to_add:
    insert_recipe(conn, r["title"], r["ingredients"], r["instructions"], r["calories"], r["prep_minutes"], r["difficulty"], r["condition_flags"])

console.print("[green]Inserted curated recipes into local DB.[/green]")


Inserted curated recipes into local DB.

In [49]:
# ----------------------------
# CELL 5 — Data Models (dataclasses)
# ----------------------------
@dataclass
class PatientProfile:
    id: str
    name: str
    age: int
    gender: str
    conditions: List[str] = field(default_factory=list)
    allergies: List[str] = field(default_factory=list)
    calorie_target: int = 1800
    max_daily_prep_minutes: int = 60
    created_at: str = field(default_factory=lambda: datetime.datetime.utcnow().isoformat())

def create_patient(conn, profile: PatientProfile):
    c = conn.cursor()
    c.execute("INSERT OR REPLACE INTO patients (id, name, age, gender, conditions, allergies, calorie_target, max_daily_prep_minutes, created_at) VALUES (?,?,?,?,?,?,?,?,?)",
              (profile.id, profile.name, profile.age, profile.gender, json.dumps(profile.conditions), json.dumps(profile.allergies), profile.calorie_target, profile.max_daily_prep_minutes, profile.created_at))
    conn.commit()

def get_patient(conn, patient_id):
    c = conn.cursor()
    c.execute("SELECT id, name, age, gender, conditions, allergies, calorie_target, max_daily_prep_minutes, created_at FROM patients WHERE id=?", (patient_id,))
    row = c.fetchone()
    if not row: return None
    return PatientProfile(id=row[0], name=row[1], age=row[2], gender=row[3], conditions=json.loads(row[4]), allergies=json.loads(row[5]), calorie_target=row[6], max_daily_prep_minutes=row[7], created_at=row[8])


In [50]:
# ----------------------------
# CELL 6 — Pantry Engine (CRUD)
# ----------------------------
def add_pantry_item(conn, patient_id, item, quantity="1", notes=""):
    c = conn.cursor()
    pid = str(uuid.uuid4())
    c.execute("INSERT INTO pantry (id, patient_id, item, quantity, notes, added_at) VALUES (?,?,?,?,?,?)",
              (pid, patient_id, item, quantity, notes, datetime.datetime.utcnow().isoformat()))
    conn.commit()
    return pid

def list_pantry(conn, patient_id):
    c = conn.cursor()
    c.execute("SELECT item, quantity, notes FROM pantry WHERE patient_id=?", (patient_id,))
    return [{"item":r[0],"quantity":r[1],"notes":r[2]} for r in c.fetchall()]


In [51]:
# ----------------------------
# CELL 7 — Medical Constraint Checker (rule engine)
# ----------------------------
# Define conservative rules for hepatitis A and general safe-recovery diet
RECOVERY_RULES = {
    "hepatitis_a": {
        "avoid": ["alcohol", "raw seafood", "fried foods", "heavy creams", "spicy chilies"],
        "recommend": ["boiled/steamed lean protein", "soft fruits", "low-fat", "hydration", "small frequent meals"],
        "notes": "Avoid hepatotoxins, heavy fats, and raw seafood. Favor low-fat, easily digestible meals."
    }
}

def check_recipe_against_conditions(recipe_row, patient_profile: PatientProfile):
    """
    recipe_row: sqlite row (id, title, ingredients, instructions, calories, prep_minutes, difficulty, condition_flags)
    Returns: tuple (allowed:bool, reasons:list)
    """
    reasons = []
    # load condition flags from recipe
    cflags = json.loads(recipe_row[7]) if recipe_row[7] else []
    # if patient has hepatitis_a -> ensure recipe flags are compatible
    if "hepatitis_a" in patient_profile.conditions:
        # if recipe contains flags that mark it unsafe - we mark as not allowed
        unsafe_flags = set(["fried_food", "high_fat", "raw_food"])
        if any(f in unsafe_flags for f in cflags):
            reasons.append("Recipe contains flags considered unsafe for hepatitis A recovery.")
        # Also check ingredients for banned items (simple substring match)
        ing_list = json.loads(recipe_row[2])
        lower_ings = " ".join(ing.lower() for ing in ing_list)
        for banned in RECOVERY_RULES["hepatitis_a"]["avoid"]:
            if banned in lower_ings:
                reasons.append(f"Contains '{banned}' which is avoided during recovery.")
    # allergies check
    for a in patient_profile.allergies:
        ing_list = json.loads(recipe_row[2])
        if any(a.lower() in ing.lower() for ing in ing_list):
            reasons.append(f"Contains allergy: {a}")
    allowed = (len(reasons)==0)
    return allowed, reasons


In [52]:
# ----------------------------
# CELL 8 — Recipe Lookup & Match (Pantry-aware)
# ----------------------------
def query_recipes(conn, patient_profile: PatientProfile, pantry_items: List[str], max_prep=None, max_calories=None):
    c = conn.cursor()
    q = "SELECT id, title, ingredients, instructions, calories, prep_minutes, difficulty, condition_flags FROM recipes"
    rows = c.execute(q).fetchall()
    results = []
    pantry_lower = [p.lower() for p in pantry_items]
    for row in rows:
        title = row[1]
        ing_list = json.loads(row[2])
        # pantry match score = fraction of ingredients present
        present = sum(1 for ing in ing_list if any(p in ing.lower() for p in pantry_lower))
        score = present / max(1, len(ing_list))
        # check constraints
        allowed, reasons = check_recipe_against_conditions(row, patient_profile)
        if not allowed:
            continue
        if max_prep and row[5] > max_prep: continue
        if max_calories and row[4] > max_calories: continue
        results.append({"id":row[0],"title":title,"ingredients":ing_list,"instructions":row[3],"calories":row[4],"prep_minutes":row[5],"difficulty":row[6],"score":score})
    # sort by pantry match (score) then low prep
    results.sort(key=lambda r:(-r["score"], r["prep_minutes"]))
    return results


In [53]:
# ----------------------------
# CELL 9 — Planner Agent (deterministic, multi-step)
# ----------------------------
def build_daily_plan(conn, patient_id, day_date=None, meals_per_day=3):
    patient = get_patient(conn, patient_id)
    if not patient:
        raise ValueError("Patient not found")
    # get pantry items
    pantry = list_pantry(conn, patient_id)
    pantry_items = [p["item"] for p in pantry]
    # target calories per meal
    target_total = patient.calorie_target
    per_meal = max(300, target_total // meals_per_day)
    # find candidate recipes
    candidates = query_recipes(conn, patient, pantry_items, max_prep=patient.max_daily_prep_minutes, max_calories=per_meal+200)
    # Greedy assignment: choose best-scoring recipe per meal, avoid duplicates
    chosen = []
    used_ids = set()
    for i in range(meals_per_day):
        for cand in candidates:
            if cand["id"] in used_ids: continue
            # score bias: prefer close to per_meal calories and match pantry
            cal_diff = abs(cand["calories"] - per_meal)
            score = cand["score"] - (cal_diff/1000.0)
            cand["_score"] = score
        # pick top candidate not used
        candidates_sorted = sorted([c for c in candidates if c["id"] not in used_ids], key=lambda x: -x["_score"])
        if not candidates_sorted:
            break
        pick = candidates_sorted[0]
        chosen.append(pick)
        used_ids.add(pick["id"])
    # build plan JSON
    plan = {
        "id": str(uuid.uuid4()),
        "patient_id": patient_id,
        "date": (day_date or datetime.date.today().isoformat()),
        "meals": [
            {"slot": i+1, "recipe_id": r["id"], "title": r["title"], "calories": r["calories"], "prep_minutes": r["prep_minutes"], "instructions": r["instructions"], "ingredients": r["ingredients"]} for i,r in enumerate(chosen)
        ],
        "total_calories": sum(r["calories"] for r in chosen),
        "created_at": datetime.datetime.utcnow().isoformat()
    }
    # Persist
    c = conn.cursor()
    c.execute("INSERT INTO plans (id, patient_id, plan_json, doctor_approved, created_at) VALUES (?,?,?,?,?)", (plan["id"], patient_id, json.dumps(plan), 0, plan["created_at"]))
    conn.commit()
    return plan

# Helper to pretty-print a plan
def print_plan(plan):
    console.rule(f"Meal Plan for {plan['date']}")
    table = Table()
    table.add_column("Slot")
    table.add_column("Recipe")
    table.add_column("Calories")
    table.add_column("Prep min")
    for m in plan["meals"]:
        table.add_row(str(m["slot"]), m["title"], str(m["calories"]), str(m["prep_minutes"]))
    console.print(table)
    console.print(f"[bold]Total calories:[/bold] {plan['total_calories']}")


In [54]:
# ----------------------------
# CELL 10 — Doctor Approval Workflow
# ----------------------------
def get_pending_plans(conn, patient_id=None):
    c = conn.cursor()
    q = "SELECT id, patient_id, plan_json, doctor_approved, created_at FROM plans WHERE doctor_approved=0"
    if patient_id:
        q = q + " AND patient_id=?"
        rows = c.execute(q, (patient_id,)).fetchall()
    else:
        rows = c.execute(q).fetchall()
    return [{"id":r[0],"patient_id":r[1], "plan":json.loads(r[2]), "doctor_approved":r[3], "created_at":r[4]} for r in rows]

def approve_plan(conn, plan_id, approved=True, doctor_notes=""):
    c = conn.cursor()
    c.execute("UPDATE plans SET doctor_approved=?, plan_json=? WHERE id=?", (1 if approved else 0, json.dumps({"approved": approved, "notes": doctor_notes}), plan_id))
    conn.commit()


In [55]:
# ----------------------------
# CELL 11 — Shopping List Generator
# ----------------------------
def generate_shopping_list(conn, plan):
    # aggregate ingredients from plan meals, subtract pantry items
    pantry = list_pantry(conn, plan["patient_id"])
    pantry_items = [p["item"].lower() for p in pantry]
    needed = {}
    for m in plan["meals"]:
        for ing in m["ingredients"]:
            key = ing.lower()
            if any(p in key for p in pantry_items):
                continue
            needed[key] = needed.get(key, 0) + 1
    # return simple list
    return list(needed.keys())


In [56]:
# ----------------------------
# CELL 12 — Memory (session-level store)
# ----------------------------
def add_memory(conn, patient_id, key, value):
    c = conn.cursor()
    mid = str(uuid.uuid4())
    c.execute("INSERT INTO memory (id, patient_id, key, value, created_at) VALUES (?,?,?,?,?)", (mid, patient_id, key, value, datetime.datetime.utcnow().isoformat()))
    conn.commit()
    return mid

def search_memory(conn, patient_id, query):
    c = conn.cursor()
    rows = c.execute("SELECT key, value FROM memory WHERE patient_id=?", (patient_id,)).fetchall()
    # naive keyword matching
    results = []
    for k,v in rows:
        if query.lower() in (k.lower() + " " + v.lower()):
            results.append({"key":k,"value":v})
    return results


In [57]:
# ----------------------------
# CELL 13 — Optional: Simple Gemini Hook (if user provided GOOGLE_API_KEY)
# ----------------------------
# This is optional: if GOOGLE_API_KEY is present, you may choose to call Gemini for explanation text or more natural phrasing.
# We won't use ADK to avoid external heavy deps; we will call the REST-like Gemini via google-genai if available.
USE_GEMINI = False
if GOOGLE_API_KEY:
    # user opt-in: set USE_GEMINI True if you want natural language elaborations
    USE_GEMINI = False  # Keep False by default; change to True if you want to enable calls
    if USE_GEMINI:
        try:
            # Attempt minimal import for google genai SDK
            import google.auth
            from google.oauth2 import service_account
            from google.api_core.client_options import ClientOptions
            # NOTE: full Gemini integration in Kaggle may require extra setup; left as optional.
            console.print("[green]Gemini integration requested — ensure environment and permissions are set.[/green]")
        except Exception as e:
            console.print(f"[red]Gemini integration unavailable: {e}[/red]")
            USE_GEMINI = False


In [58]:
# ----------------------------
# CELL 14 — Demo: Create sample patient and add pantry items
# ----------------------------
# Create patient profile for demo
pid = str(uuid.uuid4())
profile = PatientProfile(id=pid, name="Demo Patient", age=30, gender="F", conditions=["hepatitis_a"], allergies=[], calorie_target=1600, max_daily_prep_minutes=60)
create_patient(conn, profile)
# Add pantry items: rice, chicken, carrots
add_pantry_item(conn, pid, "rice")
add_pantry_item(conn, pid, "chicken breast")
add_pantry_item(conn, pid, "carrots")

rprint("[green]Created demo patient and added pantry items: rice, chicken breast, carrots[/green]")


Created demo patient and added pantry items: rice, chicken breast, carrots

In [59]:
# ----------------------------
# CELL 15 — Demo: Generate a Plan and Show Shopping List
# ----------------------------
plan = build_daily_plan(conn, pid, meals_per_day=3)
print_plan(plan)
shopping = generate_shopping_list(conn, plan)
rprint("[bold]Shopping list (missing items):[/bold]", shopping)


──────────────────────────────────────────── Meal Plan for 2025-11-29 ─────────────────────────────────────────────

┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Slot ┃ Recipe                                                     ┃ Calories ┃ Prep min ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ 1    │ Plain Boiled Rice with Steamed Carrots and Poached Chicken │ 450      │ 30       │
│ 2    │ Steamed Fish (mild) with Mashed Potato                     │ 360      │ 25       │
│ 3    │ Oatmeal with Banana and Honey (soft)                       │ 320      │ 10       │
└──────┴────────────────────────────────────────────────────────────┴──────────┴──────────┘

Total calories: 1130

Shopping list (missing items):
[
    'salt (small pinch)',
    'water',
    'white fish fillet (100g)',
    'potato (1)',
    'butter (small 1 tsp)',
    'salt (tiny)',
    'rolled oats (1/2 cup)',
    'water or milk (1 cup)',
    'banana (1 small mashed)',
    'honey (1 tsp)'
]

In [60]:
# ----------------------------
# CELL 16 — Demo: Doctor review simulation
# ----------------------------
pending = get_pending_plans(conn, pid)
rprint(f"[bold]Pending plans for patient {pid}:[/bold] {len(pending)}")
for p in pending:
    rprint(f"Plan ID: {p['id']} — Date: {p['plan']['date']} — Meals: {len(p['plan']['meals'])}")
    # Simulate doctor approval with note
    approve_plan(conn, p['id'], approved=True, doctor_notes="Looks fine for recovery; monitor fat intake.")
rprint("[green]Simulated doctor approval done.[/green]")


Pending plans for patient 369b3e31-1887-4500-a456-630147c0f0e1: 1

Plan ID: 87f31c0f-17b7-445b-a57b-ca8c2c6766c2 — Date: 2025-11-29 — Meals: 3

Simulated doctor approval done.

In [61]:
# ----------------------------
# CELL 17 — Demo: Memory usage and retrieval
# ----------------------------
add_memory(conn, pid, "preference:temperature", "prefers warm meals")
add_memory(conn, pid, "note:likes_bananas", "patient likes bananas")
rprint("[cyan]Memory search for 'banana':[/cyan]", search_memory(conn, pid, "banana"))


Memory search for 'banana':
[{'key': 'note:likes_bananas', 'value': 'patient likes bananas'}]

In [62]:
# ----------------------------
# CELL 18 — Evaluation & Observability (simple)
# ----------------------------
# Evaluate planner: ensure returned calories <= target + margin
def evaluate_plan(conn, plan, patient_profile):
    total_cal = plan["total_calories"]
    ok = total_cal <= (patient_profile.calorie_target + 300)
    return {"total_calories": total_cal, "target": patient_profile.calorie_target, "pass": ok}

eval_result = evaluate_plan(conn, plan, profile)
rprint("[bold]Evaluation result:[/bold]", eval_result)


Evaluation result:
{'total_calories': 1130, 'target': 1600, 'pass': True}

In [63]:
# ----------------------------
# CELL 19 — Save plan export (JSON) for submission
# ----------------------------
out_file = f"mealcheckai_plan_{pid}.json"
with open(out_file, "w") as f:
    json.dump(plan, f, indent=2)
rprint(f"[green]Saved plan JSON to {out_file} — include this in submission.[/green]")


Saved plan JSON to mealcheckai_plan_369b3e31-1887-4500-a456-630147c0f0e1.json — include this in submission.

In [64]:
# ----------------------------
# CELL 21 — Final instructions to the evaluator and how to run
# ----------------------------
console.rule("[bold]MealCheckAI - How to run & verify (short) [/bold]")
print(textwrap.dedent("""
1) Run all cells in the notebook top-to-bottom in a Kaggle Notebook.
2) Optional: Add GOOGLE_API_KEY as a secret if you want to implement Gemini callouts (advanced).
3) Cells 14-16 demonstrate a full patient flow:
   - create patient & pantry
   - generate daily meal plan (planner agent)
   - doctor approval simulation
   - memory & shopping list.
4) The output JSON file mealcheckai_plan_<patient_id>.json is saved (CELL 19) — include in submission.
5) If you want to adapt the planner to other conditions, add condition keywords and new recipes to the recipes table (CELL 4).
6) To deploy externally, you can wrap planner endpoints in FastAPI / Cloud Run / Agent Engine as desired (deployment steps documented in project write-up).

Deliverables included:
 - code (this notebook)
 - local SQLite DB (mealcheckai.db)
 - sample plan JSON (saved above)
"""))


─────────────────────────────────── MealCheckAI - How to run & verify (short)  ────────────────────────────────────


1) Run all cells in the notebook top-to-bottom in a Kaggle Notebook.
2) Optional: Add GOOGLE_API_KEY as a secret if you want to implement Gemini callouts (advanced).
3) Cells 14-16 demonstrate a full patient flow:
   - create patient & pantry
   - generate daily meal plan (planner agent)
   - doctor approval simulation
   - memory & shopping list.
4) The output JSON file mealcheckai_plan_<patient_id>.json is saved (CELL 19) — include in submission.
5) If you want to adapt the planner to other conditions, add condition keywords and new recipes to the recipes table (CELL 4).
6) To deploy externally, you can wrap planner endpoints in FastAPI / Cloud Run / Agent Engine as desired (deployment steps documented in project write-up).

Deliverables included:
 - code (this notebook)
 - local SQLite DB (mealcheckai.db)
 - sample plan JSON (saved above)

